In [22]:
import praw
import pandas as pd
import datetime as dt
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
from gensim.models import KeyedVectors, Word2Vec
from gensim.test.utils import common_texts, get_tmpfile
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
import string
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

#### Reddit OAuth

In [10]:
reddit = praw.Reddit(client_id='QglgyO3lXXhvsw',
                     client_secret='4C04cO2Cy9AlWpXQDnsa4BPhka0',
                     password='Skyguitar1',
                     user_agent='TherapyNow',
                     username='MachineLearningGuru')

#### Collect data from r/SuicideWatch

In [11]:
suicide_sub = reddit.subreddit('SuicideWatch')

In [12]:
suicide_sub = list(suicide_sub.top(limit=1000))

In [13]:
topics_dict = { "title":[], "suicidal":[] }

In [14]:
for submission in suicide_sub:
    topics_dict["title"].append(submission.title)
    topics_dict["suicidal"].append(1)

#### Collect data from r/All

In [15]:
all_sub = reddit.subreddit('All')

In [16]:
all_sub = list(all_sub.top(limit=5000))

In [17]:
for submission in all_sub:
    topics_dict["title"].append(submission.title)
    topics_dict["suicidal"].append(0)

#### Store data in csv

In [18]:
topics_data = pd.DataFrame(topics_dict)

In [19]:
topics_data.to_csv('Data.csv', index=False) 

### Model

In [23]:
emb_model = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

In [24]:
def phrase_pre_processing(phrase):
    # Basic Text PreProcessing
    phrase = re.sub(' +', ' ', re.sub(r'\d+', '', phrase.lower()).translate(str.maketrans("","", string.punctuation+'\n'+'\r'+'\t')))
    
    # Tokenizing
    tokens = word_tokenize(phrase)
    
    # Remove Stop Words
    stop_words = set(stopwords.words('english'))
    tokens = [i for i in tokens if not i in stop_words]
    
    # Lemmatization
    lemmatizer=WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(i) for i in tokens]

    return tokens

In [25]:
def phrase_to_embedding(tokens):
    words = [word for word in tokens if word in emb_model.vocab]
    embeddings = np.zeros(300)
    if len(words) >= 1:  
        avg = np.mean(emb_model[words], axis=0)
        for idx, e in enumerate(avg):
            embeddings.flat[idx] = e
            
    return embeddings

In [27]:
X = []
y = []

for index, row in topics_data.iterrows():
    tokens = phrase_pre_processing(row['title'])
    embeddings = phrase_to_embedding(tokens)
    y.append(row['suicidal'])
    X.append(embeddings)

X = np.array(X)    
y = np.array(y)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1560, 300), (391, 300), (1560,), (391,))

In [29]:
train_X,valid_X,train_label,valid_label = train_test_split(X_train, y_train, test_size=0.2, random_state=1234)
train_X.shape,valid_X.shape,train_label.shape,valid_label.shape

((1248, 300), (312, 300), (1248,), (312,))

In [77]:
input_dim = X_train.shape[1]  # Number of features
batch_size = 64
epochs = 20

input_dim

300

In [98]:
model = Sequential()
model.add(layers.Dense(32, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [102]:
model.compile(loss=binary_crossentropy, optimizer=Adam(), metrics=['accuracy'])

In [103]:
model_train = model.fit(train_X, train_label, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(valid_X, valid_label))

Train on 1248 samples, validate on 312 samples
Epoch 1/20
1248/1248 [==============================] - 1s 496us/sample - loss: 0.2791 - accuracy: 0.8814 - val_loss: 0.4091 - val_accuracy: 0.8013
Epoch 2/20
1248/1248 [==============================] - 0s 49us/sample - loss: 0.2715 - accuracy: 0.8870 - val_loss: 0.3957 - val_accuracy: 0.8237
Epoch 3/20
1248/1248 [==============================] - 0s 61us/sample - loss: 0.2686 - accuracy: 0.8894 - val_loss: 0.4069 - val_accuracy: 0.8109
Epoch 4/20
1248/1248 [==============================] - 0s 72us/sample - loss: 0.2642 - accuracy: 0.8926 - val_loss: 0.3972 - val_accuracy: 0.8205
Epoch 5/20
1248/1248 [==============================] - 0s 67us/sample - loss: 0.2577 - accuracy: 0.8990 - val_loss: 0.4051 - val_accuracy: 0.8141
Epoch 6/20
1248/1248 [==============================] - 0s 72us/sample - loss: 0.2564 - accuracy: 0.8982 - val_loss: 0.4022 - val_accuracy: 0.8205
Epoch 7/20
1248/1248 [==============================] - 0s 51us/sample

In [104]:
test_eval = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', test_eval[0])
print('Test Accuracy:', test_eval[1])

Test Loss: 0.3863633121066081
Test Accuracy: 0.8567775


In [1]:
test_phrase = ""

tokens = phrase_pre_processing(test_phrase)
embeddings = phrase_to_embedding(tokens)

print("Suicidal Ideation Probability: ", model.predict(embeddings.reshape(-1,300))[0][0])

NameError: name 'phrase_pre_processing' is not defined